![scrna5/6](https://img.shields.io/badge/scrna5/6-lightgrey)
[![Jupyter Notebook](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/scrna4.ipynb)
[![lamindata](https://img.shields.io/badge/Source%20%26%20report%20on%20LaminHub-mediumseagreen)](https://lamin.ai/laminlabs/lamindata/record/core/Transform?uid=Qr1kIHvK506rz8)

# Train an ML model on a collection

In the [previous tutorial](scrna3), we loaded an entire collection into memory to perform a simple analysis.

Here, we'll iterate over the artifacts within the collection to train an ML model.

In [ ]:
import lamindb as ln
import anndata as ad
import numpy as np

In [ ]:
ln.track()

## Preprocessing

Let us get our collection:

In [ ]:
collection_v2 = ln.Collection.filter(
    name="My versioned scRNA-seq collection", version="2"
).one()
collection_v2

## PyTorch DataLoader

If you need to train your model on a list of artifacts, you can use {meth}`~lamindb.Collection.mapped` with the PyTorch `DataLoader`.

It only loads batches into memory and thus allows to work with very large collections.

In [ ]:
from torch.utils.data import DataLoader, WeightedRandomSampler

Let us create a {class}`~lamindb.dev.MappedCollection` object from {class}`~lamindb.Collection`.

Under-the-hood, it performs a virtual inner join of the variables of the underlying `AnnData` objects.

In [ ]:
ds_mapped = collection_v2.mapped(label_keys=["cell_type"])

The intersected variable names can be accessed:

In [ ]:
len(ds_mapped.var_joint)

This is compatible with pytorch `DataLoader` because it implements `__getitem__` over a list of backed `AnnData` objects.

In [ ]:
ds_mapped[5]

The `labels` are encoded into integers.

In [ ]:
ds_mapped.encoders

Let us use a weighted sampler:

In [ ]:
# label_key for weight doesn't have to be in labels on init
sampler = WeightedRandomSampler(
    weights=ds_mapped.get_label_weights("cell_type"), num_samples=len(ds_mapped)
)
dl = DataLoader(ds_mapped, batch_size=128, sampler=sampler)

We can now iterate through the data loader:

In [ ]:
for batch in dl:
    pass

Close the connections in `MappedCollection`:

In [ ]:
ds_mapped.close()

:::{dropdown} In practice, use a context manager

```
with ds_train.mapped(label_keys=["cell_type"]) as ds_mapped:
    sampler = WeightedRandomSampler(
        weights=ds_mapped.get_label_weights("cell_type"), num_samples=len(ds_mapped)
    )
    dl = DataLoader(ds_mapped, batch_size=128, sampler=sampler)
    for batch in dl:
        pass
```
:::